# Desafio: Consumo de Dados para Previsão do Tempo das Cidades do Vale do Paraíba.

## Objetivo

Avaliar conhecimentos nas linguagens Python e SQL e na engine de processamento Apache Spark.

## Descrição

Neste desafio, você desenvolverá um notebook que será responsável por extrair dados de previsão do tempo das cidades do Vale do Paraíba, região onde se localiza a Dataside. Para consultar todas as cidades dessa região, utilizaremos a API do IBGE. No caso, basta realizar uma requisição HTTP com o método GET, utilizando a URL abaixo:

```
https://servicodados.ibge.gov.br/api/v1/localidades/mesorregioes/3513/municipios
```

Com esses dados, gerar um data frame e a partir dele uma temp view. Ex: "cities"

Utilizando os nomes das cidades, deverão ser consultados os dados de previsão de tempo para cada cidade. Para realizar essa consulta, poderá ser utilizada qualquer uma das APIs informadas no link abaixo.

[Public APIs - Wather](https://github.com/public-apis/public-apis#weather)

Obs.: Para algumas, pode ser necessário cadastrar-se para acessar sua API Key. Mas nenhuma delas deve precisar cadastrar cartão de crédito ou adicionar qualquer valor monetário para utilizar. Caso alguma solicite, basta optar por outra.

Com os dados consultados, gerar um data frame e partir dele outra temp view. Ex: "forecasts"

Com as temp views geradas, utilizar Spark SQL para criar queries e gerar data frames das seguintes tabelas:

- Tabela 1: dados de previsão do tempo para os próximos cinco dias, para cada data e cidade consultadas. As colunas dessa tabela serão:
    - Cidade
    - CodigoDaCidade
    - Data
    - Regiao
    - Pais
    - Latitude
    - Longigute
    - TemperaturaMaxima
    - TemperaturaMinima
    - TemperaturaMedia
    - VaiChover
    - ChanceDeChuva
    - CondicaoDoTempo
    - NascerDoSol
    - PorDoSol
    - VelocidadeMaximaDoVento
    
    Obs.: Os valores da coluna "VaiChover" deverá ser "Sim" ou "Não". E a coluna "CodigoDaCidade" é o ID retornado junto com os nomes da cidades na API do IBGE.
    Obs.: Dependendo da API utilizada, algumas colunas podem não existir e ficarão em branco. Você deve optar por uma API que traga o maior número de informações possível.

- Tabela 2: quantidade de dias com chuva e sem chuva para os dias consultados, para cada data consultada. Colunas:
    - Cidade
    - QtdDiasVaiChover
    - QtdDiasNaoVaiChover
    - TotalDiasMapeados

Essas tabelas deverão ser exportadas em formado CSV e entregue no final do desafio.

## To Do

[ ] - Consultar municípios do Vale do Paraíba, gerar um data frame e criar uma temp view com esses dados.
[ ] - Consultar dados do tempo para cada município, gerar um data frame e criar uma outra temp view.
[ ] - Utilizar Spark SQL para gerar os data frames das Tabelas 1 e 2.
[ ] - Exportar os data frames para CSV.

## Atenção

- Existe um limite de requisições de 10000 requests por conta cadastrada na m3o.
- Essa API pode retornar cidades de outras regiões que possuem nome semelhante a alguma cidade do Vale do Paraiba. Pode mantê-las ou filtrar para gerar as tabelas apenas com dados de Regiao = Sao Paulo. Fica a seu critério.

## Entregando o desafio

Concluindo todos os passos informados em To Do, basta salvar o arquivo .ipynb do notebook e enviar para a Dataside juntamente com os CSVs das duas tabelas.


In [37]:
import findspark
findspark.init()

import requests
import json
import unidecode
from pyspark.sql import SparkSession
import pandas as pd 


spark = SparkSession.builder \
      .master("local[1]") \
      .appName("SparkByExamples.com") \
      .getOrCreate()

In [38]:
# Buscar cidades do Vale do Paraíba
#Consulta API

nome_cidade = []
cidade_Vale = []
resposta = requests.get("https://servicodados.ibge.gov.br/api/v1/localidades/mesorregioes/3513/municipios")
informacoes = resposta.json()
print (informacoes)
colunas = ["cidades"]
for i in informacoes:
    #busca o nome da cidade
    for j in i:
        if (j == 'nome'):
            nome_cidade = i[j]  
        #microrregião  
        if j == 'microrregiao':
            for k in i[j]:
                #mesoregiao
                if k == 'mesorregiao':
                    for l in i[j][k]:
                        if i[j][k][l]=='Vale do Paraíba Paulista':
                            #achou a cidade do vale do paraíba
                            #grava cidade em lista
                            cidade_Vale.append(nome_cidade)  

#criar dataframe com a coluna cidades 
df = pd.DataFrame(data={"cidades": cidade_Vale})
df.to_csv("./Arquivos/cidades.csv", sep=',', index=False, encoding='latin1')
           
# Criar data frame com as cidades

# Criar view com as cidades



[{'id': 3502507, 'nome': 'Aparecida', 'microrregiao': {'id': 35051, 'nome': 'Guaratinguetá', 'mesorregiao': {'id': 3513, 'nome': 'Vale do Paraíba Paulista', 'UF': {'id': 35, 'sigla': 'SP', 'nome': 'São Paulo', 'regiao': {'id': 3, 'sigla': 'SE', 'nome': 'Sudeste'}}}}, 'regiao-imediata': {'id': 350052, 'nome': 'Guaratinguetá', 'regiao-intermediaria': {'id': 3511, 'nome': 'São José dos Campos', 'UF': {'id': 35, 'sigla': 'SP', 'nome': 'São Paulo', 'regiao': {'id': 3, 'sigla': 'SE', 'nome': 'Sudeste'}}}}}, {'id': 3503158, 'nome': 'Arapeí', 'microrregiao': {'id': 35052, 'nome': 'Bananal', 'mesorregiao': {'id': 3513, 'nome': 'Vale do Paraíba Paulista', 'UF': {'id': 35, 'sigla': 'SP', 'nome': 'São Paulo', 'regiao': {'id': 3, 'sigla': 'SE', 'nome': 'Sudeste'}}}}, 'regiao-imediata': {'id': 350053, 'nome': 'Cruzeiro', 'regiao-intermediaria': {'id': 3511, 'nome': 'São José dos Campos', 'UF': {'id': 35, 'sigla': 'SP', 'nome': 'São Paulo', 'regiao': {'id': 3, 'sigla': 'SE', 'nome': 'Sudeste'}}}}}, {

In [54]:
#pegar da API
# for t in cidade_Vale:
#      city = {'city_name':t+',SP'}
#      previsao = requests.get("https://api.hgbrasil.com/weather?key=220a140c", params=city)
#      informacoes_tempo = previsao.json()
#      resultados = informacoes_tempo.pop('results')
#      temperatura = resultados.pop('forecast')
#      print(temperatura)

with open('forecast.json') as f:
    file = json.load(f)
    print(file)


#print(type(dados))
#print(forecast)
#df1 = pd.DataFrame([elemento])
#df1.to_csv("./Arquivos/temperaturas.csv", sep=',', index=False, encoding='latin1')   
    
    #resultados = informacoes_tempo.pop('results')
    #temperatura = resultados.pop('forecast')
    #temperaturas.append(temperatura)



#for u in temperaturas,cidade_Vale:
 #   print (temperaturas)
    #df = pd.DataFrame(data={temperaturas})
    #df.to_csv("./Arquivos/forecast.csv", sep=',', index=False, encoding='latin1')

    #print (temperatura)
    #cidades= t
    #df1 = spark.read.json(informacoes_tempo)
    #display(df1)
 
#df = pd.DataFrame(data={cidades:temperatura})
#df.to_csv("./Arquivos/forecast.csv", sep=',', index=False, encoding='latin1')

# Criar view com as previsões
# TODO

JSONDecodeError: Extra data: line 2 column 1 (char 1165)

In [ ]:
# Criar DF da Tabela 1
# TODO

In [ ]:
# Criar DF da Tabela 2
# TODO

In [ ]:
# Exportar CSVs
# TODO